In [1]:
%load_ext autoreload
%autoreload 2

from gwak.train.dataloader import SignalDataloader
from pathlib import Path
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import lightning.pytorch as pl
import yaml
from tqdm import tqdm
from ml4gw.transforms import SpectralDensity, Whiten
from sklearn.metrics import roc_curve, auc
from tqdm import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

/home/katya.govorkova/gwak2/gwak/data/prior.py:5: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


In [2]:
data_dir = "/home/katya.govorkova/gwak2/gwak/output/O4_MDC_background/HL/"
sample_rate = 4096
kernel_length = 0.5
psd_length = 64
fduration = 1
fftlength = 2
batch_size = 1024
batches_per_epoch = 500
num_workers = 2
data_saving_file = None
duration = fduration + kernel_length

from gwak.data.prior import SineGaussianBBC, LAL_BBHPrior, GaussianBBC, CuspBBC, KinkBBC, KinkkinkBBC, WhiteNoiseBurstBBC
from ml4gw.waveforms import SineGaussian, IMRPhenomPv2, Gaussian, GenerateString, WhiteNoiseBurst

signal_classes = [
    "SineGaussian",
    "BBH",
    "Gaussian",
    "Cusp",
    "Kink",
    "KinkKink",
    "WhiteNoiseBurst",
    "Background"
]
priors = [
    SineGaussianBBC(),
    LAL_BBHPrior(),
    GaussianBBC(),
    CuspBBC(),
    KinkBBC(),
    KinkkinkBBC(),
    WhiteNoiseBurstBBC(),
    None
]
waveforms = [
    SineGaussian(
        sample_rate=sample_rate,
        duration=duration
    ),
    IMRPhenomPv2(),
    Gaussian(
        sample_rate=sample_rate,
        duration=duration
    ),
    GenerateString(
        sample_rate=sample_rate
    ),
    GenerateString(
        sample_rate=sample_rate
    ),
    GenerateString(
        sample_rate=sample_rate
    ),
    WhiteNoiseBurst(
        sample_rate=sample_rate,
        duration=duration
    ),
    None
]
extra_kwargs = [
    None,
    {"ringdown_duration":0.9},
    None,
    None,
    None,
    None,
    None,
    None
]

In [3]:
loader = SignalDataloader(signal_classes,
    priors,
    waveforms,
    extra_kwargs,
    data_dir=data_dir,
    sample_rate=sample_rate,
    kernel_length=kernel_length,
    psd_length=psd_length,
    fduration=fduration,
    fftlength=fftlength,
    batch_size=batch_size,
    batches_per_epoch=batches_per_epoch,
    num_workers=num_workers,
    data_saving_file=data_saving_file
)
test_loader = loader.test_dataloader()

/home/katya.govorkova/miniconda3/envs/gwak/lib/python3.11/site-packages/ml4gw/dataloading/hdf5_dataset.py:83: ContiguousHdf5Warning: File /home/katya.govorkova/gwak2/gwak/output/O4_MDC_background/HL/background-1401696321-8160.h5 contains datasets that were generated without using chunked storage. This can have severe performance impacts at data loading time. If you need faster loading, try re-generating your datset with chunked storage turned on.
  warnings.warn(


In [4]:
for batch in test_loader:
    [batch] = batch
    waveforms, params, ras, decs, phics = loader.generate_waveforms(batch.shape[0])
    batch = batch.to(device)
    x,snr = loader.multiInject_SNR(waveforms, batch)
    labels = torch.cat([(i+1)*torch.ones(loader.num_per_class[i]) for i in range(loader.num_classes)])
    break

In [ ]:
from gwak.train.cl_models import Crayon
import yaml

ckpt = "/home/eric.moreno/gwak2/gwak/output/S4_SimCLR_multiSignalAndBkg/47-2400.ckpt"
cfg_path = "/home/eric.moreno/gwak2/gwak/output/S4_SimCLR_multiSignalAndBkg/config.yaml"
with open(cfg_path,"r") as fin:
    cfg = yaml.load(fin,yaml.FullLoader)

embed_model = Crayon(**cfg['model']['init_args'])
ckpt_dict = torch.load(ckpt, map_location='cuda')
embed_model.load_state_dict(ckpt_dict['state_dict'])
embed_model.eval()
embed_model.to("cuda")

metric_model_path = "/home/katya.govorkova/gwak2/gwak/output/S4_SimCLR_multiSignalAndBkg_NF_onlyBkg/model.pt"
metric_model = torch.jit.load(metric_model_path)
metric_model.to("cuda")
metric_model.eval()


# Containers to store anomaly scores and binary labels
all_scores = []
all_binary_labels = []
all_labels = []
all_snrs = []
all_embeddings = []

# Iterate over the test data loader
niter = 1
for ib in range(niter):
    print(f"iter {ib+1}/{niter}")
    for batch in tqdm(test_loader):
        [batch] = batch
        # Generate the corresponding waveforms
        waveforms, params, ras, decs, phics = loader.generate_waveforms(batch.shape[0])
        batch = batch.to(device)
        print('loaded batch')
        # Process the waveforms into the required input format
        x, snrs = loader.multiInject_SNR(waveforms, batch)
        # snrs = snrs/sample_rate # TODO: why is this necessary lmao

        # Reconstruct the ground truth multi-class labels
        labels = torch.cat([(i+1)*torch.ones(loader.num_per_class[i]) for i in range(loader.num_classes)]).to(device)

        with torch.no_grad():
            z = embed_model.model(x.to(device)).cpu()
            scores = metric_model(z).squeeze().cpu().numpy()
            # The combined model directly outputs the anomaly score
            #scores = combined_model(x.to(device)).squeeze()  # (batch_size,)
            #scores = scores.cpu().numpy()
            # Invert the scores so that higher values correspond to more anomalous behavior
            scores = -scores

        # Convert the multi-class labels to binary labels:
        # Treat "Background" (assumed to be label 8) as 0 (normal) and all other classes as 1 (anomaly)
        binary_labels = (labels != 8).cpu().numpy()

        all_scores.append(scores)
        all_binary_labels.append(binary_labels)
        all_labels.append(labels.cpu().numpy())
        all_snrs.append(snrs.cpu().numpy())
        all_embeddings.append(z.numpy())

# Concatenate scores and labels across batches
all_scores = np.concatenate(all_scores)
all_binary_labels = np.concatenate(all_binary_labels)
all_labels = np.concatenate(all_labels)
all_snrs = np.concatenate(all_snrs)
all_embeddings = np.concatenate(all_embeddings)

iter 1/1


  0%|                                                                                                        | 0/500 [00:00<?, ?it/s]

loaded batch


  0%|▏                                                                                           | 1/500 [01:40<13:59:42, 100.97s/it]

loaded batch


...

  1%|█                                                                                            | 6/500 [08:35<11:17:46, 82.32s/it]

loaded batch


  1%|█▎                                                                                           | 7/500 [09:55<11:11:54, 81.77s/it]

loaded batch


  2%|█▍                                                                                           | 8/500 [11:21<11:21:43, 83.14s/it]

In [ ]:
all_scores = np.load("roc_data/nf_scores.npy")
all_labels = np.load("roc_data/labels.npy")
all_snrs = np.load("roc_data/snrs.npy")
all_embeddings = np.load("roc_data/embeddings.npy")
all_binary_labels = (all_labels!=8).astype(np.float32)

## try gmm

In [ ]:
import sys 
sys.path.append("/n/home11/sambt/phlab-neurips25/utils")
from plotting import make_corner
make_corner(all_embeddings,all_labels)

In [ ]:
make_corner(all_embeddings[all_labels==8],all_labels[all_labels==8])

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KernelDensity
#mix = GaussianMixture(n_components=2).fit(all_embeddings[all_labels==8])
mix = KernelDensity(kernel='gaussian',bandwidth=0.5).fit(all_embeddings[all_labels==8])

In [ ]:
signal_classes = [
    "SineGaussian",
    "BBH",
    "Gaussian",
    "Cusp",
    "Kink",
    "KinkKink",
    "WhiteNoiseBurst",
    "Background"
]
signal_labels = np.arange(len(signal_classes))+1
scores_gmm = []
for chunk in tqdm(np.array_split(all_embeddings,len(all_embeddings)//100,axis=0)):
    scores_gmm.append(-mix.score_samples(chunk))
scores_gmm = np.concatenate(scores_gmm)

In [ ]:
np.save("roc_data/scores_kde.npy",scores_gmm)

In [ ]:
plt.figure(figsize=(8,6))
bins = np.linspace(scores_gmm[all_labels==8].min(),scores_gmm[all_labels==8].max(),50)
for lab,name in zip(signal_labels,signal_classes):
    h = plt.hist(scores_gmm[all_labels==lab],label=name,histtype='step',lw=2,bins=bins)
plt.legend()
plt.yscale('log')

In [ ]:
fpr, tpr, thresholds = roc_curve((all_labels!=8).astype(np.float32), scores_gmm)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC (All Signals)")
plt.legend(loc="lower right")
plt.savefig("roc_combined_gmm.png")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']  # Colors for each anomaly class
for i, anomaly_class in enumerate(signal_classes[:-1]):  # Exclude "Background"
    # The anomaly class numeric label is i+1.
    anomaly_val = i + 1
    # Filter to only examples that are either the current anomaly class or Background (label 8).
    idx = np.where((all_labels == anomaly_val) | (all_labels == 8))[0]
    if idx.size == 0:
        continue  # skip if no examples
    scores_i = scores_gmm[idx]
    # Create binary ground truth: positive if the example is of the current anomaly class, 0 if background.
    binary_labels = (all_labels[idx] == anomaly_val).astype(int)
    
    # Compute the ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(binary_labels, scores_i)
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, color=colors[i % len(colors)], lw=2,
             label=f"{anomaly_class} (AUC = {roc_auc:.2f})")

# Plot reference line (diagonal)
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
#plt.yscale('log')
plt.title("ROC Curves by Anomaly Class")
plt.legend(loc="lower right")
plt.savefig("rocs_bySignal_gmm.png")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']  # Colors for each anomaly class

for i, anomaly_class in enumerate(signal_classes[:-1]):  # Exclude "Background"
    # The anomaly class numeric label is i+1.
    anomaly_val = i + 1
    # Filter to only examples that are either the current anomaly class or Background (label 8).
    idx = np.where((all_labels == anomaly_val) | (all_labels == 8))[0]
    if idx.size == 0:
        continue  # skip if no examples
    scores_i = scores_gmm[idx]
    # Create binary ground truth: positive if the example is of the current anomaly class, 0 if background.
    binary_labels = (all_labels[idx] == anomaly_val).astype(int)
    
    # Compute the ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(binary_labels, scores_i)
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, color=colors[i % len(colors)], lw=2,
             label=f"{anomaly_class} (AUC = {roc_auc:.2f})")

# Plot reference line (diagonal)
plt.plot(np.linspace(0,1,100), np.linspace(0,1,100), color="navy", lw=2, linestyle="--")
plt.xlim([1e-3, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
#make x axis log
plt.xscale('log')

plt.title("ROC Curves by Anomaly Class")
plt.legend(loc="lower right")
plt.savefig("rocs_bySignal_logX_gmm.png")
plt.show()

In [ ]:
h = plt.hist(scores_gmm[all_labels==8],bins=100)
plt.yscale('log')

In [ ]:
Ns = 10
toy_scores = []
for i in tqdm(range(Ns)):
    samps,_ = mix.sample(1_000_000)
    toy_scores.append(-mix.score_samples(samps))
toy_scores = np.concatenate(toy_scores)

In [ ]:
segment_duration = 0.5
year_sec = 3.154e7
fraction_1yr = (1.0 / year_sec) * segment_duration


toy_scores = scores_gmm[all_labels==8]
q = np.quantile(toy_scores,1-fraction_1yr)

plt.figure(figsize=(8,6))
h=plt.hist(toy_scores,bins=100)
plt.yscale('log')
plt.axvline(q,lw=2,color='red',linestyle='--')
print(q)

In [ ]:
#threshold_1yr = q
threshold_1yr = 7

In [ ]:
anom_mask = (all_labels != 8)
if np.any(anom_mask):
    snr_min, snr_max = all_snrs[anom_mask].min(), all_snrs[anom_mask].max()
else:
    raise ValueError("No anomaly samples found in the test set!")

num_bins = 10
bin_edges = np.logspace(-3, 3, num_bins + 1)
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

plt.figure(figsize=(8,6))

for i, anom_class_name in enumerate(signal_classes[:-1]):
    class_label = i + 1  # classes are 1..7
    mask = (all_labels == class_label)
    if not np.any(mask):
        # If no samples for that class, skip
        continue

    class_scores = scores_gmm[mask]
    class_snrs = all_snrs[mask]

    # Bin by SNR
    bin_idx = np.digitize(class_snrs, bin_edges) - 1  # bin indices in [0..num_bins-1]

    frac_detected = []
    for b in range(num_bins):
        in_bin = (bin_idx == b)
        if not np.any(in_bin):
            frac_detected.append(np.nan)  # or 0.0 if you prefer
        else:
            # fraction that exceed threshold
            frac = np.mean(class_scores[in_bin] > threshold_1yr)
            frac_detected.append(frac)

    plt.plot(bin_centers, frac_detected, marker='o', label=anom_class_name)

plt.xlabel("SNR")
plt.ylabel("Fraction of events detected")
plt.title("Fraction of Events Detected at 1/Year FAR vs. SNR")
plt.ylim([0, 1.05])
#plt.xlim([0,1e7])
plt.xscale('log')
plt.legend()
plt.grid(True)
#plt.savefig("fraction_1overYearFAR_SNR_gmm.png")
plt.show()

## use nf scores

In [ ]:
# Compute the ROC curve and AUC using scikit-learn
fpr, tpr, thresholds = roc_curve(all_binary_labels, all_scores)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC (All Signals)")
plt.legend(loc="lower right")
plt.savefig("roc_combined.png")
plt.show()

In [ ]:
# The list of signal class names, with the last one being "Background".
signal_classes = [
    "SineGaussian",
    "BBH",
    "Gaussian",
    "Cusp",
    "Kink",
    "KinkKink",
    "WhiteNoiseBurst",
    "Background"
]

# Compute and plot ROC curves for each anomaly class (all but "Background").

plt.figure(figsize=(8,6))
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']  # Colors for each anomaly class

for i, anomaly_class in enumerate(signal_classes[:-1]):  # Exclude "Background"
    # The anomaly class numeric label is i+1.
    anomaly_val = i + 1
    # Filter to only examples that are either the current anomaly class or Background (label 8).
    idx = np.where((all_labels == anomaly_val) | (all_labels == 8))[0]
    if idx.size == 0:
        continue  # skip if no examples
    scores_i = all_scores[idx]
    # Create binary ground truth: positive if the example is of the current anomaly class, 0 if background.
    binary_labels = (all_labels[idx] == anomaly_val).astype(int)
    
    # Compute the ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(binary_labels, scores_i)
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, color=colors[i % len(colors)], lw=2,
             label=f"{anomaly_class} (AUC = {roc_auc:.2f})")

# Plot reference line (diagonal)
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
#plt.yscale('log')
plt.title("ROC Curves by Anomaly Class")
plt.legend(loc="lower right")
plt.savefig("rocs_bySignal.png")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']  # Colors for each anomaly class

for i, anomaly_class in enumerate(signal_classes[:-1]):  # Exclude "Background"
    # The anomaly class numeric label is i+1.
    anomaly_val = i + 1
    # Filter to only examples that are either the current anomaly class or Background (label 8).
    idx = np.where((all_labels == anomaly_val) | (all_labels == 8))[0]
    if idx.size == 0:
        continue  # skip if no examples
    scores_i = all_scores[idx]
    # Create binary ground truth: positive if the example is of the current anomaly class, 0 if background.
    binary_labels = (all_labels[idx] == anomaly_val).astype(int)
    
    # Compute the ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(binary_labels, scores_i)
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, color=colors[i % len(colors)], lw=2,
             label=f"{anomaly_class} (AUC = {roc_auc:.2f})")

# Plot reference line (diagonal)
plt.plot(np.linspace(0,1,100), np.linspace(0,1,100), color="navy", lw=2, linestyle="--")
plt.xlim([1e-3, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
#make x axis log
plt.xscale('log')

plt.title("ROC Curves by Anomaly Class")
plt.legend(loc="lower right")
plt.savefig("rocs_bySignal_logX.png")
plt.show()

In [ ]:
segment_duration = 0.5
year_sec = 3.154e7

fraction_1yr = (1.0 / year_sec) * segment_duration
# Extract background-only data
bg_mask = (all_labels == 8)
bg_scores = all_scores[bg_mask]
threshold_1yr = np.quantile(bg_scores,1-fraction_1yr)

h = plt.hist(bg_scores,bins=100)
plt.axvline(threshold_1yr,color='red')
plt.yscale('log')

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
# -----------------------------------------------------------------------------
# 3) BIN ANOMALIES BY SNR AND COMPUTE FRACTION DETECTED (SCORE > threshold_1yr)
# -----------------------------------------------------------------------------

# Your class names, last is "Background"
signal_classes = [
    "SineGaussian",
    "BBH",
    "Gaussian",
    "Cusp",
    "Kink",
    "KinkKink",
    "WhiteNoiseBurst",
    "Background",
]

# We'll define 10 bins in SNR from the minimum anomaly SNR to the maximum
# anomaly SNR across all anomaly classes (labels 1..7).
anom_mask = (all_labels != 8)
if np.any(anom_mask):
    snr_min, snr_max = all_snrs[anom_mask].min(), all_snrs[anom_mask].max()
else:
    raise ValueError("No anomaly samples found in the test set!")

num_bins = 10
bin_edges = np.logspace(-2, 3, num_bins + 1)
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

plt.figure(figsize=(8,6))

for i, anom_class_name in enumerate(signal_classes[:-1]):
    class_label = i + 1  # classes are 1..7
    mask = (all_labels == class_label)
    if not np.any(mask):
        # If no samples for that class, skip
        continue

    class_scores = all_scores[mask]
    class_snrs = all_snrs[mask]

    # Bin by SNR
    bin_idx = np.digitize(class_snrs, bin_edges) - 1  # bin indices in [0..num_bins-1]

    frac_detected = []
    for b in range(num_bins):
        in_bin = (bin_idx == b)
        if not np.any(in_bin):
            frac_detected.append(np.nan)  # or 0.0 if you prefer
        else:
            # fraction that exceed threshold
            frac = np.mean(class_scores[in_bin] > threshold_1yr)
            frac_detected.append(frac)

    plt.plot(bin_centers, frac_detected, marker='o', label=anom_class_name)

plt.xlabel("SNR")
plt.ylabel("Fraction of events detected")
plt.title("Fraction of Events Detected at 1/Year FAR vs. SNR")
plt.ylim([0, 1.05])
plt.xscale('log')
plt.legend()
plt.grid(True)
#plt.savefig("fraction_1overYearFAR_SNR.png")
plt.show()

In [ ]:
bins = np.logspace(1,7,100)
for i,sig in enumerate(signal_classes[:-1]):
    h=plt.hist(all_snrs[all_labels==i+1],bins=np.logspace(-3,3,100),label=sig,histtype='step')
#plt.yscale('log')
plt.xscale('log')
plt.xlabel("SNR")
plt.legend()